In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch3d as p3d
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.io import load_obj, load_ply, save_ply
from pytorch3d.loss import chamfer_distance

In [2]:
### config

# 定义点云数据的输入维度和输出维度
input_dim = 3  # 每个点的特征维度
output_dim = 3  # 输出点云的特征维度（可以与输入维度相同）

device = "cuda" if torch.cuda.is_available() else "cpu"
num_epochs = 20
batch_size = 1

sxxxxes = os.listdir("../../data/all_results/")

In [3]:
# 创建一个简单的全连接神经网络类
class PointCloudFCNet(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(PointCloudFCNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)  # 输入层到隐藏层1
        self.fc2 = nn.Linear(64, 128)  # 隐藏层1到隐藏层2
        self.fc3 = nn.Linear(128, 256)  # 隐藏层2到隐藏层3
        self.fc4 = nn.Linear(256, 128)  # 隐藏层3到隐藏层4
        self.fc5 = nn.Linear(128, output_dim)  # 隐藏层4到输出层

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)  # 输出层不使用激活函数
        return x


# DataSet类
class TrainingSet(Dataset):
    def __init__(self, s_name_list, transform=None) -> None:
        self.input = [
            f"../../data/all_results/{sname}/0_sphere.obj" for sname in s_name_list
        ]
        self.target = [
            f"../../data/all_results/{sname}/{sname}.ply" for sname in s_name_list
        ]
        self.transform = transform

    def __len__(self):
        return len(self.input)

    def __getitem__(self, index):
        return load_obj(self.input[index],load_textures=False)[0].to(device), load_ply(self.target[index])[0].to(device)
    
    def get_target(self,input):
        pass

In [4]:
def plot_pointcloud(points, title=""):
    """Sample points uniformly from the surface of the mesh."""
    x, y, z = points.clone().detach().cpu().squeeze().unbind(1)
    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(111, projection="3d")
    ax.scatter3D(x, z, -y)
    ax.set_xlabel("x")
    ax.set_ylabel("z")
    ax.set_zlabel("y")
    ax.set_title(title)
    ax.view_init(190, 30)
    plt.show()

In [5]:
mydataset = TrainingSet(sxxxxes)
data_loader = DataLoader(mydataset, batch_size=batch_size, shuffle=False)


In [6]:
# 创建模型实例
model = PointCloudFCNet(input_dim, output_dim).to(device)
model

PointCloudFCNet(
  (fc1): Linear(in_features=3, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=128, bias=True)
  (fc5): Linear(in_features=128, out_features=3, bias=True)
)

In [7]:
# 定义损失函数和优化器
criterion = chamfer_distance
optimizer = optim.Adam(model.parameters(), lr=0.001)  # 使用Adam优化器

In [9]:
# 训练模型

for epoch in range(num_epochs):
    for input_point_cloud,target_point_cloud in data_loader:
        optimizer.zero_grad()
        output_point_cloud = model(input_point_cloud)  # 前向传播
        loss,_ = criterion(output_point_cloud, target_point_cloud)  # 计算损失
        loss.backward()  # 反向传播
        optimizer.step()  # 优化模型参数

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    # if (epoch + 1) % 5 == 0:
    #     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

        # plot_pointcloud(output_point_cloud)# 绘制点云

Epoch [1/20], Loss: 0.0035
Epoch [2/20], Loss: 0.0034
Epoch [3/20], Loss: 0.0034
Epoch [4/20], Loss: 0.0032
Epoch [5/20], Loss: 0.0030
Epoch [6/20], Loss: 0.0031
Epoch [7/20], Loss: 0.0030
Epoch [8/20], Loss: 0.0029
Epoch [9/20], Loss: 0.0028
Epoch [10/20], Loss: 0.0030
Epoch [11/20], Loss: 0.0030
Epoch [12/20], Loss: 0.0029
Epoch [13/20], Loss: 0.0031
Epoch [14/20], Loss: 0.0030
Epoch [15/20], Loss: 0.0031
Epoch [16/20], Loss: 0.0029
Epoch [17/20], Loss: 0.0028
Epoch [18/20], Loss: 0.0030
Epoch [19/20], Loss: 0.0029
Epoch [20/20], Loss: 0.0028


In [10]:
# 使用训练好的模型生成新的点云数据
new_point_cloud = model(input_point_cloud) # TODO: 这里要改成用路径读一个点云
new_point_cloud
# 输出的new_point_cloud包含了经过神经网络处理后的新点云数据

tensor([[[ 0.4002, -0.2044, -0.2243],
         [ 0.5375, -0.2674, -0.2334],
         [ 0.2285, -0.1704, -0.1763],
         ...,
         [ 0.0020,  0.5497, -0.4095],
         [-0.0228,  0.5715, -0.4060],
         [-0.0902,  0.4521, -0.4407]]], device='cuda:0',
       grad_fn=<ViewBackward0>)

In [12]:
#看一眼新的输出长啥样
# save_ply("./kkwd.ply",new_point_cloud.squeeze())